# Defined functions 

In [ ]:
def label_extraction(column):
    values = column.unique().tolist()
    return {value: index for index, value in enumerate(values)}

def labeller(column, feature_name):
    """Used for Label encoding of catogrical features"""
    return column.apply(lambda x: label_extraction(column)[x])


def count_missing_values(df):
    """For showing missing values"""
    missing_value_count = {}
    for column in df.columns:
        missing_value_count[labels[column]] = df[column].isna().sum()
    
    missing_values_df = pd.DataFrame({'Feature': list(missing_value_count.keys()), 
                                      'No of values missing': list(missing_value_count.values())})
    
    return missing_values_df

def assign_values_by_percentiles(data):
    """
    Assigns values to elements in the data based on predefined percentiles and values.

    Parameters:
        data (array-like): The data for which values need to be assigned.

    Returns:
        list: A list containing the assigned values based on the percentiles.
    """
    # Define percentiles and corresponding values
    percentiles = [25, 50, 75, 100]
    values = [0.25, .5, 0.75, 1]

    # Calculate percentile values
    percentile_values = [np.percentile(data, p) for p in percentiles]

    # Create a list to store the assigned values
    assigned_values = []

    # Assign values based on percentiles
    for value in data:
        if value <= percentile_values[0]:
            assigned_values.append(values[0])
        elif value <= percentile_values[1]:
            assigned_values.append(values[1])
        elif value <= percentile_values[2]:
            assigned_values.append(values[2])
        else:
            assigned_values.append(values[3])

    return assigned_values

# Importing libraries

In [ ]:
# Importing necessary libraries for data manipulation, visualization, and model building
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
import plotly.express as px
import seaborn as sns
import shap  # Used for model interpretation with SHAP values
from math import sqrt  # For mathematical operations
import scipy.stats as stats

# Importing metrics and tools from scikit-learn for model evaluation
from sklearn.metrics import r2_score, mean_squared_error as mse
from sklearn.model_selection import train_test_split  # Splitting data into training and testing sets
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

# Models
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, AdaBoostRegressor  # Ensemble methods
from sklearn.tree import DecisionTreeRegressor  # Decision tree regressor
from sklearn.svm import SVR  # Support Vector Regressor
from sklearn.multioutput import MultiOutputRegressor  # For multi-output regression
from xgboost import XGBRegressor  # Extreme Gradient Boosting
from catboost import CatBoostRegressor  # Categorical Boosting
from lightgbm import LGBMRegressor  # Light Gradient Boosting Machine

# Hyperparameter optimization using Hyperopt
from hyperopt import hp, tpe, Trials, fmin

# Suppressing warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

In [ ]:
labels={'mt':'Materials type','btu':'Biochar type (Unmodified or modified)','bt':'Biochar type','bet':'BET surface area (m2/g)','p':'Pore volume (cm3/g)','ph':'solution pH (pHsol)','rt':'Reactor temperature','asc':'Initial As concentration (Total) mg/L','ad':'Adsorbent dosage (g/L)', 'e':'Equilibrium/Reaction time (h)','pt':'Pyrolysis temperature','target':'As(V)'}

# Preproessing Data

In [ ]:
df=pd.read_csv('data.csv')
y_final=df.target
df.drop(columns='target',inplace=True)
df.describe()

In [ ]:
for n in ['mt', 'btu', 'bt']:
    print(label_extraction(df[n]))
    df[n] = labeller(df[n], n)

In [ ]:
df.describe()

count_missing_values(df)

In [ ]:
no_missing_df=df.dropna()                                           #Data with no missing values
some_missing_df=df.drop(index=no_missing_df.index.to_list())        #Data with missing values

## Missing Value Imputation

Train-Test Split

In [ ]:
no_missing_df=no_missing_df.sort_index()
x_no_missing=no_missing_df[['mt','btu','bt','asc','pt']]
y_no_missing=no_missing_df.drop(columns=['mt','btu','bt','asc','pt'])
x_no_missing_train, x_no_missing_test, y_no_missing_train, y_no_missing_test = train_test_split(x_no_missing, y_no_missing, test_size=0.2,random_state=260)

Base Model Scores

In [ ]:
# List of regression models
models = [DecisionTreeRegressor(), RandomForestRegressor(),XGBRegressor(), MultiOutputRegressor(estimator = CatBoostRegressor(verbose=0)),MultiOutputRegressor(estimator = AdaBoostRegressor()),MultiOutputRegressor(estimator=GradientBoostingRegressor())]
results={}
for model in models:
    results[model.__class__.__name__]=fit_and_evaluate_model(model, x_no_missing_train, y_no_missing_train, x_no_missing_test, y_no_missing_test)
    accuracies = cross_val_score(estimator = model, X = x_no_missing_train, y= y_no_missing_train, cv=5)
    print(f" Mean Accuracy: {accuracies.mean()}")

Model Statistics

In [ ]:
# Fitting DecisionTreeRegressor Regression to the Training set wihtout hyperparameter tuning

model = DecisionTreeRegressor()

model.fit(x_no_missing_train, y_no_missing_train)

#Measure the R2 for training and test set
model_score = model.score(x_no_missing_train,y_no_missing_train)

y_predicted = model.predict(x_no_missing_test)

y_predicted_train = model.predict(x_no_missing_train)

print("The training R2 is: ", model.score(x_no_missing_train, y_no_missing_train))
print("The test R2 is: ", model.score(x_no_missing_test, y_no_missing_test))
print("The R2 is: ", r2_score(y_predicted,y_no_missing_test))



# The mean squared error & Variance
print("MSE: %.2f"% mse(y_no_missing_test, y_predicted))
print("RMSE of train set: %.2f"% sqrt(mse(y_no_missing_train, y_predicted_train)))
print("RMSE of test set: %.2f"% sqrt(mse(y_no_missing_test, y_predicted)))


#k-cross validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = model, X = x_no_missing_train, y= y_no_missing_train, cv=5)
print("The mean accuracy is: ", accuracies.mean())


#Plotting the joint plot of  actual v/s predicted
pp_tr = model.predict(x_no_missing_train)

Hyperparameter tuning (Decision Tree Regressor)

In [ ]:
from sklearn.model_selection import GridSearchCV
model = DecisionTreeRegressor()
param_grid = {  
	'max_features': ['sqrt', 'log2', None], 
	'max_depth': [3, 6, 9,10,11,12], 
	'max_leaf_nodes': [3, 6, 9], 
}
tuning = GridSearchCV(estimator=model, 
                          param_grid = param_grid,
                          cv=3,
                          verbose=1,
                          n_jobs=-1,
                          scoring='neg_mean_squared_error')   
tuning.fit(x_no_missing_train, y_no_missing_train)
print("Best parameters:")
print(tuning.best_params_)
random_forest_tuned_results=fit_and_evaluate_model(DecisionTreeRegressor(), x_no_missing_train, y_no_missing_train, x_no_missing_test, y_no_missing_test)

Value Imputation

In [ ]:
features=some_missing_df[['mt','btu','bt','asc','pt']]
results=model.predict(features) #predicting
results=pd.DataFrame(results,columns=['bet','p','ph','rt','ad','e'],index=features.index) #forming a DataFrame
df_complete=df.combine_first(results) #combinging with original dataset to replace missing values

In [ ]:
df_complete['target']=y_final
print(df_complete[df_complete['target'].isna()])
df=df_complete.dropna()

# Explodatory Data Analysis

Pearson correlation heatmap

In [ ]:
plt.figure(figsize=(10,10))
heatmap=sns.heatmap(df.corr(),annot=True,cmap='YlGnBu')
heatmap.set_xticklabels([labels[n] for n in df.columns if n in df.columns],rotation=90)
heatmap.set_yticklabels([labels[n] for n in df.columns if n in df.columns],rotation=0)
plt.title('Pearson Correlation Heatmap')
plt.show()

Histogram

In [ ]:
sns.set(style="whitegrid")


num_cols = df.shape[1]
num_rows = (num_cols - 1) // 3 + 1

fig, axes = plt.subplots(num_rows, 3, figsize=(15, num_rows * 5))

if num_rows == 1:
    axes = [axes]

# Loop through each column and plot histogram
for i, col in enumerate(df.columns):
    ax = axes[i // 3][i % 3]
    sns.histplot(df[col], ax=ax, kde=True, color='skyblue', bins=20)
    ax.set_title(labels[col], fontsize=14)
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.grid(False)

# Remove empty subplot(s) if any
for i in range(num_cols, num_rows * 3):
    fig.delaxes(axes[i // 3][i % 3])

# Adjust layout
plt.tight_layout()
plt.show()

# Model Building

Train-test split
70% for Training
15% for validation
15% for testing

In [ ]:
df = df.sort_index()
x = df.drop(columns='target')
y = df['target']
x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.3, random_state=42)
x_valid, x_test, y_valid, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=22)

Models Without tuning.
1. XGBregressor
2. Catboost
3. LGMregressor

In [ ]:
sns.set_theme()
plt.rcdefaults()
model = XGBRegressor()
model.fit(x_train, y_train)
model_score = model.score(x_train,y_train)
y_predicted = model.predict(x_test)
y_predicted_train = model.predict(x_train)
print("The training R2 is: ", model.score(x_train, y_train))
print("The test R2 is: ", model.score(x_test, y_test))
print("The model R2 is: ", r2_score(y_test, y_predicted))
print("MSE: %.2f"% mse(y_test, y_predicted))
print("RMSE of model: %.2f"% sqrt(mse(y_test, y_predicted)))
print("RMSE of train set: %.2f"% sqrt(mse(y_train, y_predicted_train)))
accuracies = cross_val_score(estimator = model, X = x_train, y= y_train, cv=3)
print("The mean accuracy is: ", accuracies.mean())
pp_tr = model.predict(x_train)
g = sns.JointGrid(x=y_test, y=y_predicted)
sns.scatterplot(x=y_train, y=pp_tr, s=100, color='orange', ax=g.ax_joint)
sns.scatterplot(x=y_test, y=y_predicted, s=100, color='blue', ax=g.ax_joint)
sns.regplot(x=y_test, y=y_predicted, ax=g.ax_joint)
g.set_axis_labels("Actual", "Predicted", fontsize =22, fontname = 'Times New Roman')
sns.histplot(x=y_train,ax=g.ax_marg_x, color ='orange')
sns.histplot(x=y_test, ax=g.ax_marg_x, color ='blue')

In [ ]:
model = CatBoostRegressor(verbose=0)
model.fit(x_train, y_train)
model_score = model.score(x_train,y_train)
y_predicted = model.predict(x_test)
y_predicted_train = model.predict(x_train)
print("The training R2 is: ", model.score(x_train, y_train))
print("The test R2 is: ", model.score(x_test, y_test))
print("The model R2 is: ", r2_score(y_test, y_predicted))
print("MSE: %.2f"% mse(y_test, y_predicted))
print("RMSE of model: %.2f"% sqrt(mse(y_test, y_predicted)))
print("RMSE of train set: %.2f"% sqrt(mse(y_train, y_predicted_train)))
accuracies = cross_val_score(estimator = model, X = x_train, y= y_train, cv=10)
print("The mean accuracy is: ", accuracies.mean())
pp_tr = model.predict(x_train)
g = sns.JointGrid(x=y_test, y=y_predicted)
sns.scatterplot(x=y_train, y=pp_tr, s=100, color='orange', ax=g.ax_joint)
sns.scatterplot(x=y_test, y=y_predicted, s=100, color='blue', ax=g.ax_joint)
sns.regplot(x=y_test, y=y_predicted, ax=g.ax_joint)
g.set_axis_labels("Actual", "Predicted", fontsize =22, fontname = 'Times New Roman')
sns.histplot(x=y_train,ax=g.ax_marg_x, color ='orange')
sns.histplot(x=y_test, ax=g.ax_marg_x, color ='blue')

In [ ]:
model = LGBMRegressor(verbose=-1)
model.fit(x_train, y_train)
model_score = model.score(x_train,y_train)
y_predicted = model.predict(x_test)
y_predicted_train = model.predict(x_train)
print("The training R2 is: ", model.score(x_train, y_train))
print("The test R2 is: ", model.score(x_test, y_test))
print("The model R2 is: ", r2_score(y_test, y_predicted))
print("MSE: %.2f"% mse(y_test, y_predicted))
print("RMSE of model: %.2f"% sqrt(mse(y_test, y_predicted)))
print("RMSE of train set: %.2f"% sqrt(mse(y_train, y_predicted_train)))
accuracies = cross_val_score(estimator = model, X = x_train, y= y_train, cv=3)
print("The mean accuracy is: ", accuracies.mean())
pp_tr = model.predict(x_train)
from sklearn.metrics import mean_squared_error, r2_score
g = sns.JointGrid(x=y_test, y=y_predicted)
sns.scatterplot(x=y_train, y=pp_tr, s=100, color='orange', ax=g.ax_joint)
sns.scatterplot(x=y_test, y=y_predicted, s=100, color='blue', ax=g.ax_joint)
sns.regplot(x=y_test, y=y_predicted, ax=g.ax_joint)
g.set_axis_labels("Actual", "Predicted", fontsize =22, fontname = 'Times New Roman')
sns.histplot(x=y_train,ax=g.ax_marg_x, color ='orange')
sns.histplot(x=y_test, ax=g.ax_marg_x, color ='blue')

## Tuning

1. XGBregressor (Cross-Valnidation)
2. Catboost (Bayesian)
3. LGBregressor (Cross-Valnidation)

In [ ]:
model = XGBRegressor()
param_grid = {'eta':[0.05,0.1,0.15,0.2],
                 'max_depth':[4,6,8,10],
                 'subsample':[0.5,1],
                 'colsample_bytree':[0.5,1],
                 'eval_metric':['rmse'],
                 'seed':[42],
                 'n_estimators':np.arange(100,1001,100).tolist()}

tuning = GridSearchCV(estimator=model, 
                          param_grid = param_grid,
                          cv=3,
                          verbose=1,
                          n_jobs=-1,
                          scoring='neg_mean_squared_error')
tuning.fit(x_train, y_train)
print("Best parameters:")
print(tuning.best_params_)

model = XGBRegressor(**tuning.best_params_)
model.fit(x_train, y_train)
model_score = model.score(x_train,y_train)
y_predicted = model.predict(x_test)
y_predicted_train = model.predict(x_train)
print("The training R2 is: ", model.score(x_train, y_train))
print("The test R2 is: ", model.score(x_test, y_test))
print("The model R2 is: ", r2_score(y_test, y_predicted))
print("MSE: %.2f"% mse(y_test, y_predicted))
print("RMSE of model: %.2f"% sqrt(mse(y_test, y_predicted)))
print("RMSE of train set: %.2f"% sqrt(mse(y_train, y_predicted_train)))
accuracies = cross_val_score(estimator = model, X = x_train, y= y_train, cv=3)
print("The mean accuracy is: ", accuracies.mean())
pp_tr = model.predict(x_train)
g = sns.JointGrid(x=y_test, y=y_predicted)
sns.scatterplot(x=y_train, y=pp_tr, s=100, color='orange', ax=g.ax_joint)
sns.scatterplot(x=y_test, y=y_predicted, s=100, color='blue', ax=g.ax_joint)
sns.regplot(x=y_test, y=y_predicted, ax=g.ax_joint)
g.set_axis_labels("Actual", "Predicted", fontsize =22, fontname = 'Times New Roman')
sns.histplot(x=y_train,ax=g.ax_marg_x, color ='orange')
sns.histplot(x=y_test, ax=g.ax_marg_x, color ='blue')

In [ ]:
space = {
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.5)),
    'depth': hp.quniform('depth', 4, 12, 1),
    'l2_leaf_reg': hp.loguniform('l2_leaf_reg', np.log(1), np.log(100)),
    'iterations': hp.quniform('iterations', 100, 1000, 50),
    'max_bin': hp.quniform('max_bin', 4, 255, 1),
}
def objective(params):
    model = CatBoostRegressor(**params, verbose=0)
    model.fit(x_train, y_train)
    score = mse(model.predict(x_test), y_test)**0.5
    return score
trials = Trials()
best_params = fmin(fn=objective,
                   space=space,
                   algo=tpe.suggest,
                   max_evals=50,
                   trials=trials)
print("Best hyperparameters:", best_params)
cat_boost_tuned_results=fit_and_evaluate_model(CatBoostRegressor(**best_params,verbose=False),  x_train, y_train, x_test, y_test)
model = CatBoostRegressor(**best_params,verbose=False)
model.fit(x_train, y_train)
model_score = model.score(x_train,y_train)
y_predicted = model.predict(x_test)
y_predicted_train = model.predict(x_train)
print("The training R2 is: ", model.score(x_train, y_train))
print("The test R2 is: ", model.score(x_test, y_test))
print("The model R2 is: ", r2_score(y_test, y_predicted))
print("MSE: %.2f"% mse(y_test, y_predicted))
print("RMSE of model: %.2f"% sqrt(mse(y_test, y_predicted)))
print("RMSE of train set: %.2f"% sqrt(mse(y_train, y_predicted_train)))
accuracies = cross_val_score(estimator = model, X = x_train, y= y_train, cv=3)
print("The mean accuracy is: ", accuracies.mean())
pp_tr = model.predict(x_train)
g = sns.JointGrid(x=y_test, y=y_predicted)
sns.scatterplot(x=y_train, y=pp_tr, s=100, color='orange', ax=g.ax_joint)
sns.scatterplot(x=y_test, y=y_predicted, s=100, color='blue', ax=g.ax_joint)
sns.regplot(x=y_test, y=y_predicted, ax=g.ax_joint)
g.set_axis_labels("Actual", "Predicted", fontsize =22, fontname = 'Times New Roman')
sns.histplot(x=y_train,ax=g.ax_marg_x, color ='orange')
sns.histplot(x=y_test, ax=g.ax_marg_x, color ='blue')

# Shap Analysis

In [ ]:
import shap
explainer1 = shap.Explainer(model)
shap_values1 = explainer1(x_train)
shap.plots.bar(shap_values1, max_display=15)

In [ ]:
shap.summary_plot(shap_values1, x_train)

Dumping model for Web modelling

In [ ]:
import pickle
pickle.dump(model,open('model.sav','wb'))

# Optimization (Genetic Algorithm)

In [ ]:
from geneticalgorithm import geneticalgorithm as ga

Customized implementation of a genetic algorithm using the geneticalgorithm package from the repository
https://github.com/rmsolgi/geneticalgorithm/tree/master

Refer to folder (optimization) for complete information

objective function (refer to publication)

In [ ]:
# Define the objective function
def objective(params):

    prediction=model.predict(params)
    prediction=(prediction-df.target.min())/(df.target.max()-df.target.min())

    #scaling
    rt=(params[-5]-df.rt.min())/(df.rt.max()-df.rt.min())   
    e=(params[-2]-df.e.min())/(df.e.max()-df.e.min())
    
    #score
    score = -prediction +  rt + e
    return score

Genetic algorithm function

In [ ]:
def gas(varbound):
    """
    Run genetic algorithm optimization.
    
    Parameters:
    varbound (ndarray): Array of variable boundaries for optimization.
    
    Returns:
    tuple: The final score, model prediction, and genetic algorithm report.
    """
    ga_model = ga(
        function=objective,
        dimension=11,
        variable_type_mixed=np.array(['int', 'int', 'int', 'real', 'real', 'real', 'real', 'real', 'real', 'real', 'int']),
        variable_boundaries=varbound,
        algorithm_parameters={
            'max_num_iteration': 500,
            'population_size': 200,
            'mutation_probability': 0.1,
            'elit_ratio': 0.01,
            'crossover_probability': 0.5,
            'parents_portion': 0.3,
            'crossover_type': 'uniform',
            'max_iteration_without_improv': None
        }
    )
    ga_model.run()
    final_params = ga_model.output_dict['variable']
    final_score = -model_cat.predict(final_params) + final_params[-5] + final_params[-2]
    print(f"Final Score: {final_score}, Model Prediction: {model_cat.predict(final_params)}")
    return ga_model.output_dict['variable'],final_score, model_cat.predict(final_params), ga_model.report

Running

In [ ]:
# Define the optimization function
def run_optimization(t):
    """
    Run optimization for a given t value.
    Runs the sovler 10 times to reduce randomness
    Parameters:
    t (float): Fixed value for 'Initial Asc. Conc' parameter.
    """
    varbound = np.array([
        [df.mt.min(), df.mt.max()],
        [df.btu.min(), df.btu.max()],
        [df.bt.min(), df.bt.max()],
        [df.bet.min(), df.bet.max()],
        [df.p.min(), df.p.max()],
        [df.ph.min(), df.ph.max()],
        [df.rt.min(), df.rt.max()],
        [t, t],  # 'asc' parameter fixed
        [df.ad.min(), df.ad.max()],
        [df.e.min(), df.e.max()],
        [df.pt.min(), df.pt.max()]
    ])
    
    results = {}
    for n in range(10):
        print(f"t={t}, iteration={n}", end=',')
        results[n] = gas(varbound)
        
    results_df = pd.DataFrame(results, index=['input_parameters','objective_function', 'model_prediction', 'result'])
    results_df.to_csv(f"optimizing_results_{t}.csv")

# Example usage: run optimization for the first t value
t_values = [0.6710000000000002, 2.2860000000000005, 4.993, 9.97, 10.8, 20.0, 40.0, 74.97000000000001, 100.0, 1001.07] # 10-100percentile values of Int. asc conc.
run_optimization(t_values[0])

REFER TO OPTIMIZATION.ZIP for each iteration

# Plotting

In [ ]:
labels = {
    'mt': 'M. type',
    'btu': 'BioC. type(Unmodified or modified)',
    'bt': 'B. type',
    'bet': 'BET S. area (m²/g)',
    'p': 'P. vol (cm³/g)',
    'ph': 'Sol. pH (pHsol)',
    'rt': 'Rx. temp (°C)',
    'asc': 'Int. As conc. Total (mg/L)',
    'ad': 'Ad. dosage (g/L)',
    'e': 'Rx. time (h)',
    'pt': 'P. temp (°C)',
    'target': 'As(V) (mg/g)'
}

In [ ]:
legend = {
    0: 'Clay and Silica-based Materials',              # bentonite
    1: 'Clay and Silica-based Materials',              # clay
    2: 'Aluminum and Iron-based Compounds',            # Boehmite
    3: 'Aluminum and Iron-based Compounds',            # FeOx
    4: 'Aluminum and Iron-based Compounds',            # alumipowder
    5: 'Aluminum and Iron-based Compounds',            # maghemite
    6: 'Aluminum and Iron-based Compounds',            # Hematite
    7: 'Aluminum and Iron-based Compounds',            # magnetite
    8: 'Aluminum and Iron-based Compounds',            # ironManganeseOxides
    9: 'Metal-Organic Frameworks (MOFs)',              # MOF
    10: 'Carbon-based Materials',                      # Orderedmesoporouscarbon
    11: 'Carbon-based Materials',                      # grapheneoxide-carbonnotubesaerogel
    12: 'Carbon-based Materials',                      # activatedcarbon
    13: 'Organic and Biopolymer Materials',            # calciumalgitebeads
    14: 'Carbon-based Materials',                      # activatedcarbonfiber
    15: 'Carbon-based Materials',                      # graphene
    16: 'Carbon-based Materials',                      # ricestrawbiochar
    17: 'Aluminum and Iron-based Compounds',           # goethite
    18: 'Carbon-based Materials',                      # biochar
    19: 'Aluminum and Iron-based Compounds',           # Iron-BasedSorbent
    20: 'Aluminum and Iron-based Compounds',           # Fe-Al
    21: 'Aluminum and Iron-based Compounds',           # Fe
    22: 'Aluminum and Iron-based Compounds',           # Ferric-basedlayereddoublehydroxide
    23: 'Clay and Silica-based Materials',             # MCM-41
    24: 'Clay and Silica-based Materials',             # poroussilica
    25: 'Organic and Biopolymer Materials',            # fibers(V)
    26: 'Organic and Biopolymer Materials',            # TEMPO(2,2,6,6-tetramethylpiperidine-1-oxyl)
    27: 'Organic and Biopolymer Materials',            # polyvinylalcohol
    28: 'Waste-derived Materials',                     # Eggshell
    29: 'Waste-derived Materials',                     # Wastecementpowder
    30: 'Waste-derived Materials',                     # Concretesludge
    31: 'Organic and Biopolymer Materials'             # magneticchitosannoparticle
}


Terenary Plot

In [ ]:
import plotly.express as px
import plotly.io as pio

# Assuming df is your DataFrame containing data
# Assigning color and symbol based on the legend mapping
color_and_symbol = [legend[code] for code in df.mt]

fig = px.scatter_ternary(df, 
                         a='bet', 
                         b='asc', 
                         c='rt', 
                         color=df.target, 
                         size=assign_values_by_percentiles(df.pt), 
                         symbol=color_and_symbol,
                         symbol_sequence=['circle','triangle-up','cross','star'],
                         width=1600, 
                         height=1200, 
                         opacity=0.7, 
                         color_continuous_scale='bluered')

fig.update_layout(
    font=dict(family="Arial", color="black", size=24),
    ternary=dict(
        sum=1,
        bgcolor='white',
        aaxis=dict(
            min=0,
            linewidth=2,
            ticks='outside',
            tickmode='array',
            tickvals=[0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0],
            ticktext=['0', '', '10', '', '20', '', '30', '', '40', '', '50', '', '60', '', '70', '', '80', '', '90', '', '100'],
            linecolor='black',
            title=''
        ),
        baxis=dict(
            min=0,
            linewidth=2,
            ticks='outside',
            tickmode='array',
            tickvals=[0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0],
            ticktext=['0', '', '10', '', '20', '', '30', '', '40', '', '50', '', '60', '', '70', '', '80', '', '90', '', '100'],
            linecolor='black',
            title='',
        ),
        caxis=dict(
            min=0,
            linewidth=2,
            ticks='outside',
            tickmode='array',
            tickvals=[0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0],
            ticktext=['0', '', '10', '', '20', '', '30', '', '40', '', '50', '', '60', '', '70', '', '80', '', '90', '', '100'],
            linecolor='black',
            title='' 
        )
    )
)

# Change the color of grid lines
fig.update_layout(ternary=dict(
    aaxis=dict(gridcolor='grey'),  
    baxis=dict(gridcolor='grey'),  
    caxis=dict(gridcolor='grey')   
))

# Adjusting legend position
fig.update_layout(legend=dict(
    yanchor="top",
    y=1,
    xanchor="right",
    x=0.99,title="M. type",
    font=dict(size=24)
))
fig.update_layout(coloraxis_colorbar=dict(
    title="As(V)(mg/g)",
    showticklabels=False
))
fig.update_coloraxes(showscale=True)
fig.show()

Split Violot plot

In [ ]:
temp_df=df.copy()
temp_df['btu']=temp_df['btu'].apply(lambda x: 'Unmodified' if x == 0 else 'Modified')

# Set the style using plt

plt.style.use('seaborn-v0_8-poster')

# Create the plot
fig, ax = plt.subplots(figsize=(6, 6))

# Violin plot
sns.violinplot(data=temp_df, y='target', hue='btu', split=True, inner=None,
               palette={'Unmodified': 'red', 'Modified': '#FF6600'}, alpha=.6, ax=ax,linecolor='black')

# Labels and legend
ax.set_ylabel('As(V) (mg/g)', fontdict={"family": 'Arial', 'color': 'black', 'size': 28})
ax.set_xlabel('BioC. type', fontdict={"family": 'Arial', 'color': 'black', 'size': 28})
ax.legend(title="BioC. type", title_fontsize=28, fontsize=24)
ax.tick_params(axis='both', which='major', labelsize=24)
ax.set_ylim(bottom=0)
plt.show()

Correlation Heatmap Plot

In [ ]:
t_df = df.drop(columns=['mt','btu','bt'])  # removing catogorical plot
t_df.columns = [labels[n] for n in t_df.columns]

correlation_matrix = t_df.corr()

# Create an upper triangular matrix of ones with the same shape as the correlation matrix
upper_triangular_ones = np.triu(np.ones_like(correlation_matrix))

# Replace diagonal elements with zeros
np.fill_diagonal(upper_triangular_ones, 0)

plt.figure(figsize=(8,6),dpi=300)
sns.heatmap(t_df.corr(), cmap=sns.color_palette('YlGnBu',as_cmap=True), annot=False, mask=upper_triangular_ones)
plt.show()

Bar Plots

update "asd" variable for barplot for diffirent features
update "colors" for changing color

In [ ]:
asd = ['p']
colors= "#8bd3c7"
variable = [labels[n] for n in asd]

plt.figure(figsize=(8, 8), dpi=300)

# Boxplot
boxprops = dict(facecolor=colors, edgecolor='black', linewidth=2)
whiskerprops = dict(color='black', linewidth=2)
capprops = dict(color='black', linewidth=2)
medianprops = dict(color='black', linewidth=2)
flierprops = dict(marker='o', color='black', markersize=0.00001)

# Plotting boxplot
positions = range(len(asd))
boxplot = plt.boxplot(
    x=[t_df[label] for label in variable],
    patch_artist=True,
    positions=positions,
    widths=0.4,  # Width of the box
    boxprops=boxprops,
    whiskerprops=whiskerprops,
    capprops=capprops,
    medianprops=medianprops,
    flierprops=flierprops
)

# Scatter plot for maximum values
for i, label in enumerate(variable):
    max_value = t_df[label].max()
    plt.scatter(positions[i], max_value, marker='v', color='black', s=100, zorder=3, label=f'Max {label}')

# Create legend
box_legend = plt.Rectangle((0, 0), 1, 1, fc=colors, edgecolor='black', linewidth=2, label='25%~75%')
scatter_legend = plt.Line2D([], [], marker='v', color='black', linestyle='None', markersize=10, label='Max Value')

# Plotting 25th and 75th percentiles as horizontal lines
median_legend = plt.Line2D([], [], color='black', linewidth=2, label='Median')

#plt.legend(handles=[box_legend, scatter_legend, median_legend], loc='upper left', fontsize=24,prop={'family': 'Arial'})      ##(Remove the starting # to add legend)
plt.xticks(positions, variable, fontsize=24, fontname='Arial')
plt.yticks(fontsize=24, fontname='Arial')  # Adjust y-ticks

plt.savefig('_'.join(asd) + '.png')
plt.show()

Shap Plots

In [ ]:

# The data
data = {
    "asc": 19.44,
    "ad": 14.49,
    'pt':8.05,
    'e':7.1,
    "mt": 4.93,
    "bt": 4.37,
    "bet": 4.1,
    "p": 4.05,
    "ph": 2.95,
    "btu": 2.4,
    "rt": 1.1
}

total = sum(data.values())
# Convert values to percentages
percentages = {k: (v / total) * 100 for k, v in data.items()}
sorted_percentages = dict(sorted(percentages.items(), key=lambda item: item[1], reverse=True))

# Plotting
fig, ax = plt.subplots(figsize=(8, 8))

bars = ax.barh(list(percentages.keys()), list(percentages.values()), color='#126994',alpha=1, edgecolor='black',linewidth=1)

# Add labels and title with specified font style and size
ax.set_xlabel('Percentage Contribution(%)', fontname='Arial', fontsize=28)

# Annotate each bar with its percentage value
for bar in bars:
    width = bar.get_width()
    ax.text(width + 0.6, bar.get_y() + bar.get_height()/2, f'{width:.1f}%', va='center', fontname='Arial', fontsize=20,color='#126994')
ax.set_yticklabels([labels[key] for key in list(percentages.keys())], fontname='Arial', fontsize=24)
ax.set_xlim(0, 33)
ax.invert_yaxis()
plt.show()

Confidence Interval Plot

In [ ]:
best=[193.237276
,
      198.8241923
,
      175.7726192
,
      207.2319105
,
      206.0417048
,
      217.7694266
,
      205.2294573
,
      253.2950474
,
      271.5617158
,
      247.4990662
,
      265.5606908

      ]

data=pd.read_csv('utils\\as5.csv',header=None)

lower=[]
upper=[]
for n in range(len(data.columns)):
    mean=data[n].mean()
    std=data[n].std()
    lower.append(data[n].max() - 2.262 * std) #2.626 calculted from z-distriution function
    upper.append(data[n].max() + 2.262 * std)

plt.figure(figsize=(8, 8), dpi=300)
for i in range(len(best)):
    plt.plot(best[:i+1], color='red', linewidth=2)
    plt.fill_between(range(i+1), lower[:i+1], upper[:i+1], color='#ffe4e1', alpha=0.5)
    plt.ylabel(labels['target'], fontdict={"family": 'Arial', 'size': 24})
    plt.xlabel(labels['asc'], fontdict={"family": 'Arial', 'size': 24})
    
    # Set x-tick labels with custom formatting
    x_ticks_labels = [f'{n*10} percentile' for n in range(1, 11)]
    x_ticks_labels.append('Complete Range')
    plt.xticks(range(len(x_ticks_labels)), x_ticks_labels, fontsize=20, fontname='Arial', rotation=90)
    plt.yticks(fontsize=20, fontname='Arial')
plt.show()

Prediction Interval plot

In [ ]:
x_valid=pd.read_csv('utils\\x_valid.csv')
y_valid=pd.read_csv('utils\\y_valid.csv')
x_test=pd.read_csv('utils\\x_test.csv')
y_test=pd.read_csv('utils\\y_test.csv')
y_pred=model.predict(x_valid)
y_pred_cat=pd.read_csv('utils\\cat_boost_tuned_results.csv',header=None)

upper=y_pred_cat+np.percentile(y_pred-y_valid.target, 95)
lower=y_pred_cat-np.percentile(y_pred-y_valid.target, 95)


# Sorting y_test and corresponding upper, lower, y_pred_test_2
y_test_sorted = y_test.sort_values(by='target').reset_index(drop=True)
upper_sorted = pd.Series(upper[0]).sort_values().reset_index(drop=True)
lower_sorted = pd.Series(lower[0]).sort_values().reset_index(drop=True)
y_pred_test_sorted = pd.Series(y_pred_cat[0]).sort_values().reset_index(drop=True)

# Create the figure and axis
plt.figure(figsize=(8, 8), dpi=300)
ax = plt.gca()

# Remove the top and right spines

# Plotting the actual values
sns.lineplot(x=y_test_sorted.index, y=y_test_sorted['target'], ax=ax, label='True', color='red', alpha=1)

# Fill between the upper and lower confidence bounds
plt.fill_between(y_test_sorted.index, upper_sorted, lower_sorted, alpha=0.3, color='grey', label='Prediction Interval')

# Set labels
ax.set_ylabel('As(III) (mg/g)', fontdict={"family": 'Arial', 'color': 'black', 'size': 24})
ax.set_xlabel('Test Samples', fontdict={"family": 'Arial', 'color': 'black', 'size': 24})
plt.tick_params(axis='both', which='major', labelsize=16)
# Set legend
ax.legend(title="", title_fontsize=20, fontsize=16, loc='upper right')

# Show the plot
plt.show()